<span style="font-size: 14.44px;">1. Minden egyes szobát hányszor foglaltak</span>

- Csak a szoba azonosítója és a foglalások darabszáma jelenjen meg
- Azok a szobák is jelenjenek meg, ahol nem történt foglalás

In [6]:
SELECT SZOBA_ID, COUNT(*) AS 'DB'
FROM Szoba sz RIGHT JOIN Foglalas f ON sz.SZOBA_ID = f.SZOBA_FK
GROUP BY SZOBA_ID

(197 rows affected)

Total execution time: 00:00:00.484

SZOBA_ID,DB
23,3
46,4
192,2
69,4
92,6
115,7
138,7
161,6
29,6
169,6


2\. Azon vendégek adatai, akik a legtöbb férőhelyesz szobákat foglalták

- Csak klímás szoba jöhet számításba

In [16]:
SELECT v.*
FROM Vendeg v JOIN Foglalas f ON v.USERNEV = f.UGYFEL_FK
                JOIN Szoba sz ON sz.SZOBA_ID = f.SZOBA_FK
WHERE sz.SZOBA_ID IN (
    SELECT TOP 1 SZOBA_ID
    FROM Szoba
    WHERE KLIMAS = 'i'
    ORDER BY FEROHELY DESC
)

(3 rows affected)

Total execution time: 00:00:00.288

USERNEV,NEV,EMAIL,SZAML_CIM,SZUL_DAT
JOZSEFG,Gyuris József,józsef.gyuris@mail.hu,2660 Balassagyarmat Petőfi utca 1/2.,1975-05-26
PETERB,Berendi Péter,péter.berendi@mail.hu,3980 Sátoraljaújhely Vasút utca 4/10.,1969-01-01
tunde,Turcsik Tünde,tunde@gmail.com,7130 Tolna Fő út 122.,1974-02-12


Sorszámozza a vendégeket a foglalások száma alapján

    A sorszám mellett jelenjen meg a vendég azonosítója, neve és a foglalások száma <span style="color: #0000ff;">is</span>

    A legtöbbet foglaló vendég kapja az egyes sorszámot

    Holtverseny esetén a számozás annyit ugorjon, ahány egyforma érték van

In [7]:
SELECT v.USERNEV, NEV, RANK() OVER (PARTITION BY v.NEV ORDER BY COUNT(*)), COUNT(*)
FROM Vendeg v JOIN Foglalas f on v.USERNEV= f.UGYFEL_FK

: Msg 8120, Level 16, State 1, Line 1
Column 'Vendeg.USERNEV' is invalid in the select list because it is not contained in either an aggregate function or the GROUP BY clause.

Total execution time: 00:00:00.005

Készítsen lekérdezést, amely megjeleníti, hogy szálláshelyenként, azon belül a pótágyak száma szerint hány főre foglaltak összesen (felnőtt + gyermek)!

- Jelenjenek meg a részösszegek és a végösszeg is!

In [5]:
SELECT IIF(GROUPING(SZALLAS_NEV)= 1, 'Végösszeg', SZALLAS_NEV), IIF(GROUPING(sz.POTAGY)= 3, 'Részösszeg', POTAGY), SUM(FELNOTT_SZAM)+ SUM(GYERMEK_SZAM) AS 'Db'
FROM Szallashely szh JOIN Szoba sz ON szh.SZALLAS_ID = sz.SZALLAS_FK
                        JOIN Foglalas f ON f.SZOBA_FK = sz.SZOBA_ID
GROUP BY ROLLUP(SZALLAS_NEV, sz.POTAGY)

(41 rows affected)

Total execution time: 00:00:00.232

(No column name),(No column name),Db
Családi Ház,0,10
Családi Ház,1,31
Családi Ház,2,11
Családi Ház,NULL,52
Fortuna Apartman,0,18
Fortuna Apartman,1,52
Fortuna Apartman,2,36
Fortuna Apartman,NULL,106
Fortuna Panzió,0,158
Fortuna panzió,1,113


A Szoba tábla Ferohely oszlopára végezzük el a következő számításokat:

- Különböző értékek száma
- Minimum, maximum, átlag
- Null és Not null értékek százalékos aránya

In [66]:
SELECT COUNT(DISTINCT FEROHELY) AS 'db', MIN(FEROHELY) AS 'Min', MAX(FEROHELY) AS 'Max', COUNT(*)/COUNT(FEROHELY) AS 'Nem null', 
1- COUNT(*)/COUNT(FEROHELY) AS 'Null'
FROM Szoba

(1 row affected)

Total execution time: 00:00:00.130

db,Min,Max,Nem null,Null
4,1,4,1,0
